In [1]:
import os
from pathlib import Path

testfolder = str(Path().resolve().parent / 'TEMP' /  'sofia')

if not os.path.exists(testfolder):
    os.makedirs(testfolder)
    
print ("Your simulation will be stored in %s" % testfolder)

Your simulation will be stored in C:\Users\sayala\Documents\GitHub\bifacial_radiance\docs\TEMP\sofia


In [2]:
import bifacial_radiance as br
import numpy as np
import pandas as pd

In [3]:
# This information helps with debugging and getting support :)
import sys, platform
print("Working on a ", platform.system(), platform.release())
print("Python version ", sys.version)
print("Pandas version ", pd.__version__)
print("bifacial_radiance version ", br.__version__)

Working on a  Windows 10
Python version  3.11.4 | packaged by Anaconda, Inc. | (main, Jul  5 2023, 13:38:37) [MSC v.1916 64 bit (AMD64)]
Pandas version  2.1.0
bifacial_radiance version  0.4.2+161.g1692460.dirty


In [4]:
simulationname = 'tutorial_1'

# Location:
lat = 39.7407 # ° N, 
lon = -105.1686 # ° W


# Scene Parameters:
azimuth_ang=90 # Facing south
tilt = 30 # tilt.

# MakeModule Parameters
moduletype='test-module'
numpanels = 1  # AgriPV site has 3 modules along the y direction
module_x = 2 # m
module_y = 1 # m. slope we will measure
sensorsy=2
sensorsground=5

# SceneDict Parameters
pitch = 6 # m
albedo = 0.2  #'grass'     # ground albedo
clearance_height = 0.5 # m  
nMods = 4 # six modules per row.
nRows = 3  # 3 row


In [5]:
demo = br.RadianceObj(simulationname,path = testfolder)  
demo.setGround(albedo) 
epwfile = demo.getEPW(lat, lon) # NJ lat/lon 40.0583° N, 74.4057


path = C:\Users\sayala\Documents\GitHub\bifacial_radiance\docs\TEMP\sofia
Loading albedo, 1 value(s), 0.200 avg
1 nonzero albedo values.
Getting weather file: USA_CO_Golden-NREL.724666_TMY3.epw
 ... OK!


In [6]:
module=demo.makeModule(name=moduletype,x=module_x,y=module_y)


Module Name: test-module
Module test-module updated in module.json
Pre-existing .rad file objects\test-module.rad will be overwritten



In [7]:
#Valid options: mm_dd, mm_dd_HH, mm_dd_HHMM, YYYY-mm-dd_HHMM
metdata = demo.readWeatherFile(epwfile, coerce_year=2021, starttime='2021-06-01', endtime='2021-06-30')

8760 line in WeatherFile. Assuming this is a standard hourly WeatherFile for the year for purposes of saving Gencumulativesky temporary weather files in EPW folder.
Coercing year to 2021
Filtering dates
Saving file EPWs\metdata_temp.csv, # points: 8760
Calculating Sun position for Metdata that is right-labeled  with a delta of -30 mins. i.e. 12 is 11:30 sunpos


In [12]:
demo.genCumSky()  
#demo.gendaylit(timeindex=0)  

Loaded  EPWs\metdata_temp.csv
message: There were 463 sun up hours in this climate file
Total Ibh/Lbh: 0.000000


'skies\\cumulative.rad'

In [13]:
sceneDict = {'tilt':tilt,'pitch': pitch,'clearance_height':clearance_height,'azimuth':azimuth_ang, 
             'nMods': nMods, 'nRows': nRows}  
scene = demo.makeScene(module=moduletype, sceneDict=sceneDict) 


In [14]:
octfile = demo.makeOct()

Created tutorial_1.oct


If desired, you can view the Oct file at this point:

***rvu -vf views\front.vp -e .01 tutorial_1.oct***

In [15]:
analysis = br.AnalysisObj(octfile, demo.name)  
frontscan, backscan, groundscan = analysis.moduleAnalysis(scene, sensorsy=sensorsy, sensorsground=sensorsground)


y1 ! 0.0 0.0


In [16]:
analysis.analysis(octfile, simulationname+"_groundscan_East", groundscan, backscan)  # compare the back vs front irradiance  

Linescan in process: tutorial_1_groundscan_East_Row2_Module2_Front
Linescan in process: tutorial_1_groundscan_East_Row2_Module2_Back
FrontDict is:  None


(None,
 {'Wm2': [28642.95, 29353.5],
  'x': [0.1438376, -0.1448376],
  'y': [8.807511e-18, -8.868743e-18],
  'z': [0.6658006, 0.8324673],
  'r': [28642.95, 29353.5],
  'g': [28642.95, 29353.5],
  'b': [28642.95, 29353.5],
  'mattype': ['a1.1.a0.test-module.2310', 'a1.1.a0.test-module.2310'],
  'title': 'tutorial_1_groundscan_East_Row2_Module2_Back'})

In [ ]:
# Loop for sensorsground and see resolution effect 